In [1]:
%pip install pyannote.audio==3.3
%pip install numpy==1.26

  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached antlr4-python3-runtime-4.9.3.tar.gz (117 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-2.3.3-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 26.8 MB/s eta 0:00:00
Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 32.2 MB/s eta 0:00:00
Using cached numpy-2.3.3-cp312-cp312-macosx_14_0_arm64.whl (5.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.4 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144591 sha256=01e66b1c56b42277f6f2d29c925493a0f2c4188ce78cbafadfb9f7b3a5622183
  Stored in directory: /Users/yeonsigjang/Library/Caches/pip/wheels/1f/be/48/13754633f1d08d1fbfc60d5e80ae1e5d73295004

In [2]:
import os
os.environ["PATH"] += os.pathsep + r"/opt/homebrew/opt/ffmpeg@7/bin"

In [4]:
# instantiate the pipeline
from pyannote.audio import Pipeline
import torch
os.environ["HF_TOKEN"] = os.getenv("HUGGINGFACE_USE_AUTH_TOKEN")

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=os.environ["HF_TOKEN"],
)

if torch.backends.mps.is_available():
  pipeline.to(torch.device("mps"))
  print('mps is available')
else:
  print('no accelerator available')

/Users/yeonsigjang/ideaProject/gpt_agent_2025_book_pip/.venv/lib/python3.12/site-packages/pyannote/audio/pipelines/speaker_verification.py:43: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import (


mps is available


In [6]:
diarization = pipeline("./audio/싼기타_비싼기타.mp3")

# dump the diarization output to disk using RTTM format
with open("싼기타_비싼기타.rttm", "w", encoding='utf-8') as rttm:
  diarization.write_rttm(rttm)

/Users/yeonsigjang/ideaProject/gpt_agent_2025_book_pip/.venv/lib/python3.12/site-packages/torchaudio/_backend/soundfile_backend.py:71: UserWarning: The MPEG_LAYER_III subtype is unknown to TorchAudio. As a result, the bits_per_sample attribute will be set to 0. If you are seeing this warning, please report by opening an issue on github (after checking for existing/closed ones). You may otherwise ignore this warning.
  warnings.warn(
/Users/yeonsigjang/ideaProject/gpt_agent_2025_book_pip/.venv/lib/python3.12/site-packages/torchaudio/_backend/soundfile_backend.py:71: UserWarning: The MPEG_LAYER_III subtype is unknown to TorchAudio. As a result, the bits_per_sample attribute will be set to 0. If you are seeing this warning, please report by opening an issue on github (after checking for existing/closed ones). You may otherwise ignore this warning.
  warnings.warn(
/Users/yeonsigjang/ideaProject/gpt_agent_2025_book_pip/.venv/lib/python3.12/site-packages/torchaudio/_backend/soundfile_backen

In [9]:
import pandas as pd
rttm_path = "./싼기타_비싼기타.rttm"

df_rttm = pd.read_csv(
    rttm_path,
    sep=' ',
    header=None,
    names=['type', 'file', 'chnl', 'start', 'duration', 'C1', 'C2', 'speaker_id', 'C3', 'C4']
)

display(df_rttm)

,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
83,SPEAKER,싼기타_비싼기타,1,414.481,2.970,NaN,NaN,SPEAKER_00,NaN,NaN
84,SPEAKER,싼기타_비싼기타,1,417.755,3.476,NaN,NaN,SPEAKER_01,NaN,NaN
85,SPEAKER,싼기타_비싼기타,1,423.644,0.776,NaN,NaN,SPEAKER_00,NaN,NaN
86,SPEAKER,싼기타_비싼기타,1,424.741,3.527,NaN,NaN,SPEAKER_00,NaN,NaN


In [10]:
df_rttm['end'] = df_rttm['start'] + df_rttm['duration']

display(df_rttm)

,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4,end
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN,6.798
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN,11.388
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN,16.686
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN,27.875
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN,30.204
...,...,...,...,...,...,...,...,...,...,...,...
83,SPEAKER,싼기타_비싼기타,1,414.481,2.970,NaN,NaN,SPEAKER_00,NaN,NaN,417.451
84,SPEAKER,싼기타_비싼기타,1,417.755,3.476,NaN,NaN,SPEAKER_01,NaN,NaN,421.231
85,SPEAKER,싼기타_비싼기타,1,423.644,0.776,NaN,NaN,SPEAKER_00,NaN,NaN,424.420
86,SPEAKER,싼기타_비싼기타,1,424.741,3.527,NaN,NaN,SPEAKER_00,NaN,NaN,428.268


In [11]:
df_rttm["number"] = None
df_rttm.at[0, "number"] = 0

display(df_rttm)

,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4,end,number
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN,6.798,0
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN,11.388,None
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN,16.686,None
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN,27.875,None
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN,30.204,None
...,...,...,...,...,...,...,...,...,...,...,...,...
83,SPEAKER,싼기타_비싼기타,1,414.481,2.970,NaN,NaN,SPEAKER_00,NaN,NaN,417.451,None
84,SPEAKER,싼기타_비싼기타,1,417.755,3.476,NaN,NaN,SPEAKER_01,NaN,NaN,421.231,None
85,SPEAKER,싼기타_비싼기타,1,423.644,0.776,NaN,NaN,SPEAKER_00,NaN,NaN,424.420,None
86,SPEAKER,싼기타_비싼기타,1,424.741,3.527,NaN,NaN,SPEAKER_00,NaN,NaN,428.268,None


In [12]:
for i in range(1, len(df_rttm)):
  if df_rttm.at[i, "speaker_id"] != df_rttm.at[i-1, "speaker_id"]:
    df_rttm.at[i, "number"] = df_rttm.at[i-1, "number"] + 1
  else:
    df_rttm.at[i, "number"] = df_rttm.at[i-1, "number"]

display(df_rttm.head(10))

,type,file,chnl,start,duration,C1,C2,speaker_id,C3,C4,end,number
0,SPEAKER,싼기타_비싼기타,1,0.993,5.805,NaN,NaN,SPEAKER_01,NaN,NaN,6.798,0
1,SPEAKER,싼기타_비싼기타,1,7.405,3.983,NaN,NaN,SPEAKER_01,NaN,NaN,11.388,0
2,SPEAKER,싼기타_비싼기타,1,11.759,4.927,NaN,NaN,SPEAKER_01,NaN,NaN,16.686,0
3,SPEAKER,싼기타_비싼기타,1,17.210,10.665,NaN,NaN,SPEAKER_01,NaN,NaN,27.875,0
4,SPEAKER,싼기타_비싼기타,1,28.668,1.536,NaN,NaN,SPEAKER_01,NaN,NaN,30.204,0
5,SPEAKER,싼기타_비싼기타,1,32.414,0.759,NaN,NaN,SPEAKER_00,NaN,NaN,33.173,1
6,SPEAKER,싼기타_비싼기타,1,33.545,3.561,NaN,NaN,SPEAKER_00,NaN,NaN,37.106,1
7,SPEAKER,싼기타_비싼기타,1,37.628,3.763,NaN,NaN,SPEAKER_00,NaN,NaN,41.391,1
8,SPEAKER,싼기타_비싼기타,1,41.611,1.097,NaN,NaN,SPEAKER_00,NaN,NaN,42.708,1
9,SPEAKER,싼기타_비싼기타,1,41.645,0.810,NaN,NaN,SPEAKER_01,NaN,NaN,42.455,2


In [13]:
df_rttm_grouped = df_rttm.groupby("number").agg(
    start=pd.NamedAgg(column="start", aggfunc="min"),
    end=pd.NamedAgg(column="end", aggfunc="max"),
    speaker_id=pd.NamedAgg(column="speaker_id", aggfunc="first")
)

display(df_rttm_grouped)

,start,end,speaker_id
number,,,
0,0.993,30.204,SPEAKER_01
1,32.414,42.708,SPEAKER_00
2,41.645,44.024,SPEAKER_01
3,45.813,67.109,SPEAKER_00
4,67.227,82.786,SPEAKER_01
5,84.659,102.564,SPEAKER_00
6,103.492,117.532,SPEAKER_01
7,119.759,138.676,SPEAKER_00
8,139.351,168.967,SPEAKER_01


In [14]:
df_rttm_grouped["duration"] = df_rttm_grouped["end"] - df_rttm_grouped["start"]
df_rttm_grouped = df_rttm_grouped.reset_index(drop=True)
display(df_rttm_grouped)

,start,end,speaker_id,duration
0,0.993,30.204,SPEAKER_01,29.211
1,32.414,42.708,SPEAKER_00,10.294
2,41.645,44.024,SPEAKER_01,2.379
3,45.813,67.109,SPEAKER_00,21.296
4,67.227,82.786,SPEAKER_01,15.559
5,84.659,102.564,SPEAKER_00,17.905
6,103.492,117.532,SPEAKER_01,14.040
7,119.759,138.676,SPEAKER_00,18.917
8,139.351,168.967,SPEAKER_01,29.616
9,170.907,192.321,SPEAKER_00,21.414


In [15]:
df_rttm_grouped.to_csv(
    "./audio/싼기타_비싼기타_rttm.csv",
    sep=',',
    index=False,
)